# Denosing performance of image averaging

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from collections import namedtuple
from skimage import io as skimage_io  # pip install scikit-image
import information_theory as IT # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
Args = namedtuple("args", "input")
args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png")

In [ ]:
GT = skimage_io.imread(args.input)

In [ ]:
mean = np.mean(GT)
zero_mean_GT = GT.astype(np.float32) - mean
print(np.mean(zero_mean_GT))

In [ ]:
min_PSNR = 60

In [ ]:
signal = np.arange(100) - 50
signal

## Simulating additive noise

### Signal-independent additive uniform noise

In [ ]:
noise = np.random.uniform(low=-10, high=10, size=signal.size).astype(np.int32)
noisy = signal + noise

In [ ]:
plt.title("signal-independet additive uniform noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.legend()
plt.show()

In [ ]:
curves = []
#sigma_index = 1
for c in range(10, 100, 10):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.uniform(low=-c, high=c, size=GT.shape).reshape(GT.shape)
        print(np.max(noise), np.min(noise))
        noisy = np.clip(a=GT.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(c, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    print()

In [ ]:
plt.title("signal-independent additive uniform noise")
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

### An example of real additive noise: quantization noise

In [ ]:
noisy = (signal/10).astype(np.int16) * 10

In [ ]:
plt.title("quantization noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.legend()
plt.show()

Quantization noise can be modeled as signal-independent additive noise.

### Signal-dependent additive uniform noise

In [ ]:
noise = np.random.uniform(low=-signal, high=signal).astype(np.int32)
noisy = signal + noise

In [ ]:
plt.title("signal-dependent additive uniform noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.legend()
plt.show()

In [ ]:
curves = []
for c in range(1, 10, 1):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.uniform(low=-2*zero_mean_GT/c, high=2*zero_mean_GT/c)
        noisy = GT + noise
        print(np.max(noise), np.min(noise))
        noisy = np.clip(a=noisy, a_min=0, a_max=255).astype(np.uint8)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(c, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    print()

In [ ]:
plt.title("signal-dependent additive uniform noise")
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

### Signal-independent additive Gaussian noise

In [ ]:
noise = np.random.normal(loc=0, scale=10, size=signal.size).astype(np.int32)
noisy = signal + noise

In [ ]:
plt.title("signal-independent additive Gaussian noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.legend()
plt.show()

In [ ]:
curves = []
#sigma_index = 1
for std_dev in range(10, 100, 10):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.normal(loc=0, scale=std_dev, size=GT.shape).reshape(GT.shape)
        print(np.max(noise), np.min(noise))
        noisy = np.clip(a=GT.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(std_dev, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    #sigma_index += 1
    #print()

In [ ]:
plt.title("signal-independent additive Gaussian noise")
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

### Signal-dependent additive uniform noise

In [ ]:
noise = np.random.normal(loc=0, scale=signal+50)
noisy = signal + noise

In [ ]:
plt.title("signal-dependent additive Gaussian noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.legend()
plt.show()

In [ ]:
curves = []
for std_dev in range(1, 10, 1):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.normal(loc=0, scale=GT/std_dev/2)
        noisy = GT + noise
        print(np.max(noise), np.min(noise))
        noisy = np.clip(a=noisy, a_min=0, a_max=255).astype(np.uint8)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(std_dev, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    #sigma_index += 1
    print()

In [ ]:
plt.title("signal-dependent additive Gaussian noise")
for i in range(len(curves)):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

## Multiplicative uniform noise

In [ ]:
noise = np.random.uniform(low=-5, high=5, size=signal.size).astype(np.int32)
noisy = signal * (1 + noise/5)
noisy2 = np.random.uniform(low=-signal/2, high=signal/2) + signal

In [ ]:
plt.title("multiplicative uniform noise")
plt.plot(signal, signal, label="signal")
plt.plot(signal, noisy, label="noisy")
plt.plot(signal, noisy2, label="noisy2")
plt.legend()
plt.show()

In [ ]:
curves = []
#sigma_index = 1
for c in range(1, 10, 1):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.uniform(-c/10, c/10, GT.shape).reshape(GT.shape)
        #print(np.max(noise), np.min(noise))
        noisy = np.clip(a=GT.astype(np.float32) * (1 + noise), a_min=0, a_max=255).astype(np.uint8)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(c, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        plt.imshow(denoised, cmap="gray")
        plt.show()
        #input()

    curves.append(curve)
    #print()

In [ ]:
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

## Multiplicative Gaussian noise

In [ ]:
x = np.arange(10)
print(np.random.normal(x, scale=1.0))
print(np.random.poisson(x))

In [ ]:
curves = []
#sigma_index = 1
max_intensity = np.max(GT)
min_intensity = np.min(GT)
dynamic_range = max_intensity - min_intensity
normalized_GT = (GT - min_intensity) / dynamic_range
#print("GT", np.max(GT), np.min(GT))
#print(normalized_GT.dtype)
for std_dev in range(10, 100, 10):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        #noisy = np.clip(a = dynamic_range * np.random.poisson(normalized_GT*_lambda)/_lambda + min_intensity,  a_min=0, a_max=255).astype(np.uint8)
        #noisy = dynamic_range * np.random.normal(normalized_GT*std_dev)/std_dev + min_intensity
        noisy = np.random.normal(loc=GT, scale=std_dev)
        print('a', np.max(noisy), np.min(noisy))
        noisy = np.clip(a = noisy, a_min=0, a_max=255)
        noisy = noisy.astype(np.uint8)
        #print('b', np.max(noisy), np.min(noisy))
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(_lambda, iters, PSNR, np.max(denoised), np.min(denoised))
        curve.append(PSNR)
        iters += 1
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    #print()

In [ ]:
curves = []
mean = 0
#sigma_index = 1
for std_dev in range(10, 40, 4):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        noise = np.random.normal(mean, std_dev, GT.shape).reshape(GT.shape)
        print(np.max(noise), np.min(noise))
        #noisy = np.clip(a=GT.astype(np.float32) + (1 + noise), a_min=0, a_max=255).astype(np.uint8)
        noisy = dynamic_range * np.random.normal(normalized_GT*_lambda)/_lambda + min_intensity
        #noisy = GT.astype(np.float32) * (1+noise)
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(std_dev, iters, PSNR)
        curve.append(PSNR)
        iters += 1
        plt.imshow(np.clip(denoised, 0, 255).astype(np.uint8), cmap="gray")
        plt.show()
        input()

    curves.append(curve)
    #sigma_index += 1
    #print()

In [ ]:
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

## Multiplicative Poisson noise

In [ ]:
curves = []
#sigma_index = 1
max_intensity = np.max(GT)
min_intensity = np.min(GT)
dynamic_range = max_intensity - min_intensity
normalized_GT = (GT - min_intensity) / dynamic_range
#print("GT", np.max(GT), np.min(GT))
#print(normalized_GT.dtype)
for _lambda in range(20, 200, 20):
    acc_denoised = np.zeros_like(GT, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        #noisy = np.clip(a = dynamic_range * np.random.poisson(normalized_GT*_lambda)/_lambda + min_intensity,  a_min=0, a_max=255).astype(np.uint8)
        noisy = dynamic_range * np.random.poisson(normalized_GT*_lambda)/_lambda + min_intensity
        #noisy = dynamic_range * np.random.normal(normalized_GT*_lambda)/_lambda + min_intensity
        #print('a', np.max(noisy), np.min(noisy))
        noisy = np.clip(a = noisy, a_min=0, a_max=255)
        noisy = noisy.astype(np.uint8)
        #print('b', np.max(noisy), np.min(noisy))
        acc_denoised += noisy
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), GT)
        print(_lambda, iters, PSNR, np.max(denoised), np.min(denoised))
        curve.append(PSNR)
        iters += 1
        plt.imshow(denoised, cmap="gray")
        plt.show()
        input()

    curves.append(curve)
    #print()

In [ ]:
for i in range(len(range(10,100,10))):
    plt.plot(curves[i], label=i)
    #print(i)
plt.legend()
plt.show()

In [ ]:
mean = 0
var = 1000
std_dev = 30
noise = np.random.normal(mean, std_dev, GT.shape).reshape(GT.shape)
noisy = np.clip(a=GT.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
IT.distortion.PSNR(noisy, GT)

In [ ]:
IT.

In [ ]:
np.max(noisy)

In [ ]:
np.min(noisy)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
denoised = cv2.fastNlMeansDenoising(noisy, None, h=33, templateWindowSize=7, searchWindowSize=9)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy, cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy, denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
np.mean(noisy)

In [ ]:
np.mean(denoised)

In [ ]:
denoised = GT

In [ ]:
denoised.dtype

In [ ]:
from skimage.metrics import structural_similarity as ssim
from scipy import stats

In [ ]:
diff = (noisy - denoised).astype(np.uint8)

In [ ]:
plt.imshow(diff, cmap="gray")

In [ ]:
_, N = ssim(noisy, diff, full=True)

In [ ]:
plt.imshow(N, cmap="gray")

In [ ]:
_, P = ssim(noisy, denoised.astype(np.uint8), full=True)

In [ ]:
plt.imshow(P, cmap="gray")

In [ ]:
quality, _ = stats.pearsonr(N.flatten(), P.flatten())

In [ ]:
quality